# Показатели каналов по жанрам
Пример расчета по жанрам тематических телеканалов

## Описание задачи и условий расчета

- Период: 01.09.2022 - 31.10.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+
- Место просмотра: Все места (Дом+Дача)
- Каналы: жанры ДЕТСКИЕ, КИНО И СЕРИАЛЫ, МУЗЫКАЛЬНЫЕ										
- Статистики: Rtg000, Rtg%, Share, AvReach000, AvReach%, AvWReach000, AvWReach%, AvMReach000, AvMReach%, ATVReach

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике жанров телекомпаний получим id нужных нам жанров: ДЕТСКИЕ, КИНО И СЕРИАЛЫ, МУЗЫКАЛЬНЫЕ
cats.get_tv_thematic()

# Это id: 1, 3, 4

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'.
# Минимум 28 дней
date_filter = [('2022-09-01', '2022-10-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал - минимум 60 минут
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Ограничиваем расчет каналами жанров ДЕТСКИЕ, КИНО И СЕРИАЛЫ, МУЗЫКАЛЬНЫЕ. Для этого используем параметр tvThematicId.
# Для расчета по всем жанрам оставьте фильтр незаполненным (None)
company_filter = 'tvThematicId IN (1,3,4)'

# Указываем список статистик для расчета
statistics = [
    'Rtg000',
    'RtgPer',
    'Share',
    'AvReach000',
    'AvReachPer',
    'AvWkReach000',
    'AvWkReachPer',
    'AvMReach000',
    'AvMReachPer',
    'ATVReach'
]

# Указываем срезы
slices = [
    'tvThematicName' #Разбиваем по жанрам
]

# Задаем условия сортировки: жанр (от а до я)
sortings = {"tvThematicName":"ASC"}

# Задаем опции расчета
options = {
    "kitId": 4, #TV Index Plus All Russia
    "totalType": "TotalChannels", #Расчет Share от Total Channels. Для расчета от Измеряемого Тематического поменять на: TotalChannelsThem
    "useNbd": False #Расчет накопленного охвата без nbd коррекции
}

## Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_timeband_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                      daytype_filter=daytype_filter, company_filter=company_filter, 
                                      time_filter=time_filter, basedemo_filter=basedemo_filter, 
                                      targetdemo_filter=targetdemo_filter, location_filter=location_filter, 
                                      slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_timeband))

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]
df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('thematic_overall_figures_06_genres')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)